# Purpose: input raw data files from Kaggle and output cleaned data file as csv format for analysis in later parts. 

This is to ensure that we do not have to redo the data cleaning steps everytime we perform data analysis

## Import packages

In [3]:
import pandas as pd
import numpy as np
import glob
import re
import os
import fitz
import math
from textblob import TextBlob
from langdetect import detect

import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Lu
[nltk_data]     Xinyi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
#read data
# cannot store data locally as cannot push to github
trump = pd.read_csv('data/hashtag_donaldtrump.csv', lineterminator='\n')
biden = pd.read_csv('data/hashtag_joebiden.csv', lineterminator='\n')

In [3]:
# import requests
# url = "https://www.kaggle.com/datasets/manchunhui/us-election-2020-tweets/download?datasetVersionNumber=19"
# r = requests.get(url)
# r.headers

In [5]:
trump.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,3.606665e+08,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
1,2020-10-15 00:00:01,1.316529e+18,"Usa 2020, Trump contro Facebook e Twitter: cop...",26.0,9.0,Social Mediaset,3.316176e+08,Tgcom24,MediasetTgcom24,Profilo ufficiale di Tgcom24: tutte le notizie...,...,1067661.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:00.373216530
2,2020-10-15 00:00:02,1.316529e+18,"#Trump: As a student I used to hear for years,...",2.0,1.0,Twitter Web App,8.436472e+06,snarke,snarke,"Will mock for food! Freelance writer, blogger,...",...,1185.0,Portland,45.520247,-122.674195,Portland,United States of America,North America,Oregon,OR,2020-10-21 00:00:00.746433060
3,2020-10-15 00:00:02,1.316529e+18,2 hours since last tweet from #Trump! Maybe he...,0.0,0.0,Trumpytweeter,8.283556e+17,Trumpytweeter,trumpytweeter,"If he doesn't tweet for some time, should we b...",...,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:01.119649591
4,2020-10-15 00:00:08,1.316529e+18,You get a tie! And you get a tie! #Trump ‘s ra...,4.0,3.0,Twitter for iPhone,4.741380e+07,Rana Abtar - رنا أبتر,Ranaabtar,"Washington Correspondent, Lebanese-American ,c...",...,5393.0,Washington DC,38.894992,-77.036558,Washington,United States of America,North America,District of Columbia,DC,2020-10-21 00:00:01.492866121


In [6]:
biden.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,user_followers_count,user_location,lat,long,city,country,continent,state,state_code,collected_at
0,2020-10-15 00:00:01,1.316529e+18,#Elecciones2020 | En #Florida: #JoeBiden dice ...,0.0,0.0,TweetDeck,3.606665e+08,El Sol Latino News,elsollatinonews,🌐 Noticias de interés para latinos de la costa...,...,1860.0,"Philadelphia, PA / Miami, FL",25.774270,-80.193660,NaN,United States of America,North America,Florida,FL,2020-10-21 00:00:00
1,2020-10-15 00:00:18,1.316529e+18,#HunterBiden #HunterBidenEmails #JoeBiden #Joe...,0.0,0.0,Twitter for iPad,8.099044e+08,Cheri A. 🇺🇸,Biloximeemaw,"Locked and loaded Meemaw. Love God, my family ...",...,6628.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:00.517827283
2,2020-10-15 00:00:20,1.316529e+18,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,0.0,0.0,Twitter Web App,3.494182e+09,Flag Waver,Flag_Wavers,NaN,...,1536.0,Golden Valley Arizona,46.304036,-109.171431,NaN,United States of America,North America,Montana,MT,2020-10-21 00:00:01.035654566
3,2020-10-15 00:00:21,1.316529e+18,@chrislongview Watching and setting dvr. Let’s...,0.0,0.0,Twitter for iPhone,8.242596e+17,Michelle Ferg,MichelleFerg4,NaN,...,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-10-21 00:00:01.553481849
4,2020-10-15 00:00:22,1.316529e+18,#censorship #HunterBiden #Biden #BidenEmails #...,1.0,0.0,Twitter Web App,1.032807e+18,the Gold State,theegoldstate,A Silicon Valley #independent #News #Media #St...,...,390.0,"California, USA",36.701463,-118.755997,NaN,United States of America,North America,California,CA,2020-10-21 00:00:02.071309132


In [7]:
trump.shape

(970919, 21)

In [8]:
biden.shape

(776886, 21)

In [9]:
trump.dtypes

created_at               object
tweet_id                float64
tweet                    object
likes                   float64
retweet_count           float64
source                   object
user_id                 float64
user_name                object
user_screen_name         object
user_description         object
user_join_date           object
user_followers_count    float64
user_location            object
lat                     float64
long                    float64
city                     object
country                  object
continent                object
state                    object
state_code               object
collected_at             object
dtype: object

In [10]:
trump.country.unique()

array(['United States of America', nan, 'United Kingdom', 'Germany',
       'United States', 'Australia', 'Costa Rica', 'Barbados', 'Cameroon',
       'The Netherlands', 'Mexico', 'Canada', 'France', 'Peru',
       'Argentina', 'South Korea', 'Finland', 'Mongolia', 'India',
       'Japan', 'Turkey', 'Switzerland', 'Iran', 'Portugal', 'Ireland',
       'Russia', 'Greece', 'Egypt', 'Ecuador', 'Colombia', 'Sri Lanka',
       'Italy', 'China', 'Malaysia', 'Indonesia', 'Venezuela', 'Thailand',
       'Sweden', 'Nepal', 'Bangladesh', 'Philippines',
       'Dominican Republic', 'New Zealand', 'Chile', 'Honduras',
       'Azerbaijan', 'Georgia', 'Spain', 'Cyprus', 'Morocco',
       'Netherlands', 'Saudi Arabia', 'Norway', 'Singapore', 'Cuba',
       'Niger', 'Hungary', 'Greenland', 'United Arab Emirates', 'Brazil',
       'Uganda', 'Guatemala', 'Pakistan', 'Taiwan', 'Qatar',
       'Afghanistan', 'South Africa', 'Nigeria', 'Slovakia', 'Guyana',
       'Kenya', 'Iraq', 'Denmark', 'Uruguay', 'Be

In [28]:
def CleanData(data):
    df = data.copy()
    
    #only filter for english tweets
    for index, row in df.iterrows():
        try:
            language = detect(row['tweet'])
        except:
            language = 'error'
        df.loc[index, 'tweet_lang'] = language
#     df['tweet_lang'] = df['tweet'].apply(lambda x: detect(x))
    df = df[df['tweet_lang'] == 'en']
    
    #drop rows with NA country as we want to analyse by country
    df = df.dropna(subset=['country'])

    #convert column to datetime types
    date_cols = ['created_at', 'user_join_date', 'collected_at']
    numeric_cols = ['tweet_id', 'likes', 'retweet_count', 'user_id', 'user_followers_count']
    for col in date_cols:
        df[col] = pd.to_datetime(df[col])
    for col in numeric_cols:
        df[col] = df[col].astype(int)
    
    #clean up country names
    df['country'] = df['country'].replace('United States of America', 'United States')
    df = df.reset_index(drop = True)
    
    #clean up tweets
    #remove punctuation and numbers
    punctuation = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    for index, row in df.iterrows():
        tweet = row['tweet']
        tweet = ''.join([char for char in tweet if char not in punctuation])
        tweet = re.sub('[0-9]+', '', tweet)
        df.loc[index, 'tweet_new'] = tweet
    
    #tokenization
    df['tweet_new'] = df['tweet_new'].apply(lambda x: re.split('\W+', x.lower()))
    
    #remove stopwords
    stopwords = nltk.corpus.stopwords.words('english')
    df['tweet_new'] = df['tweet_new'].apply(lambda x: [word for word in x if word not in stopwords])
    
    #stemming and lemmatization
    ps = nltk.PorterStemmer()
    wn = nltk.WordNetLemmatizer()
    df['tweet_new'] = df['tweet_new'].apply(lambda x: [ps.stem(word) for word in x])
    df['tweet_new'] = df['tweet_new'].apply(lambda x: [wn.lemmatize(word) for word in x])
    return df

In [29]:
sample_trump = trump.iloc[:10000, :]
sample_trump.shape

(10000, 21)

In [30]:
trump_cleaned = CleanData(sample_trump)
trump_cleaned.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,lat,long,city,country,continent,state,state_code,collected_at,tweet_lang,tweet_new
0,2020-10-15 00:00:02,-2147483648,"#Trump: As a student I used to hear for years,...",2,1,Twitter Web App,8436472,snarke,snarke,"Will mock for food! Freelance writer, blogger,...",...,45.520247,-122.674195,Portland,United States,North America,Oregon,OR,2020-10-21 00:00:00.746433060,en,"[trump, student, use, hear, year, ten, year, h..."
1,2020-10-15 00:00:08,-2147483648,You get a tie! And you get a tie! #Trump ‘s ra...,4,3,Twitter for iPhone,47413798,Rana Abtar - رنا أبتر,Ranaabtar,"Washington Correspondent, Lebanese-American ,c...",...,38.894992,-77.036558,Washington,United States,North America,District of Columbia,DC,2020-10-21 00:00:01.492866121,en,"[get, tie, get, tie, trump, ralli, iowa, https..."
2,2020-10-15 00:00:17,-2147483648,@CLady62 Her 15 minutes were over long time ag...,2,0,Twitter for Android,1138416104,Farris Flagg,FarrisFlagg,#BidenHarris2020 #JoeBiden2020 #KamalaHarrisFo...,...,33.782519,-117.228648,NaN,United States,North America,California,CA,2020-10-21 00:00:01.866082651,en,"[cladi, minut, long, time, ago, omarosa, never..."
3,2020-10-15 00:00:18,-2147483648,@DeeviousDenise @realDonaldTrump @nypost There...,0,0,Twitter for iPhone,-2147483648,Stacey Gulledge 🇺🇸 Patriot ♥️ KAG 🙏 👮‍♀️♥️,sm_gulledge,"Patriot, Wife, “Shaken not Stirred” Mom of two...",...,40.225357,-82.688140,NaN,United States,North America,Ohio,OH,2020-10-21 00:00:02.612515712,en,"[deeviousdenis, realdonaldtrump, nypost, mani,..."
4,2020-10-15 00:00:20,-2147483648,One of the single most effective remedies to e...,0,0,Twitter Web App,540476889,Jamieo,jamieo33,"Don't know what I am. Can lean left and right,...",...,40.969989,-77.727883,NaN,United States,North America,Pennsylvania,PA,2020-10-21 00:00:02.985732243,en,"[one, singl, effect, remedi, erad, anoth, roun..."


In [31]:
trump_cleaned.shape

(4283, 23)

In [32]:
trump_cleaned.dtypes

created_at              datetime64[ns]
tweet_id                         int32
tweet                           object
likes                            int32
retweet_count                    int32
source                          object
user_id                          int32
user_name                       object
user_screen_name                object
user_description                object
user_join_date          datetime64[ns]
user_followers_count             int32
user_location                   object
lat                            float64
long                           float64
city                            object
country                         object
continent                       object
state                           object
state_code                      object
collected_at            datetime64[ns]
tweet_lang                      object
tweet_new                       object
dtype: object

In [33]:
trump_cleaned.country.unique()

array(['United States', 'United Kingdom', 'Germany', 'Australia',
       'Barbados', 'Cameroon', 'The Netherlands', 'Canada', 'South Korea',
       'Mongolia', 'Turkey', 'France', 'Iran', 'Portugal', 'Ireland',
       'Russia', 'Greece', 'Egypt', 'Sri Lanka', 'China', 'Malaysia',
       'Italy', 'Thailand', 'Sweden', 'India', 'Philippines',
       'New Zealand', 'Switzerland', 'Chile', 'Azerbaijan', 'Georgia',
       'Colombia', 'Argentina', 'Spain', 'Cyprus', 'Morocco', 'Japan',
       'Saudi Arabia', 'Norway', 'Singapore', 'Niger', 'Hungary',
       'Mexico', 'Greenland', 'Netherlands', 'United Arab Emirates',
       'Uganda', 'Guatemala', 'Pakistan', 'Finland', 'Peru', 'Qatar',
       'Afghanistan', 'South Africa', 'Nigeria', 'Bangladesh', 'Slovakia',
       'Guyana', 'Kenya', 'Brazil', 'Denmark', 'Indonesia', 'Lebanon',
       'Belgium', 'Liechtenstein', 'Tunisia', 'Iceland', 'Sudan',
       'Czech Republic', 'Malta', 'Israel', 'Ecuador', 'Poland',
       'Austria', 'Luxembourg', '

In [34]:
sample_biden = biden.iloc[:10000, :]
sample_biden.shape

(10000, 21)

In [35]:
biden_cleaned = CleanData(sample_biden)
biden_cleaned.head()

,created_at,tweet_id,tweet,likes,retweet_count,source,user_id,user_name,user_screen_name,user_description,...,lat,long,city,country,continent,state,state_code,collected_at,tweet_lang,tweet_new
0,2020-10-15 00:00:20,-2147483648,@IslandGirlPRV @BradBeauregardJ @MeidasTouch T...,0,0,Twitter Web App,-2147483648,Flag Waver,Flag_Wavers,NaN,...,46.304036,-109.171431,NaN,United States,North America,Montana,MT,2020-10-21 00:00:01.035654566,en,"[islandgirlprv, bradbeauregardj, meidastouch, ..."
1,2020-10-15 00:00:22,-2147483648,#censorship #HunterBiden #Biden #BidenEmails #...,1,0,Twitter Web App,-2147483648,the Gold State,theegoldstate,A Silicon Valley #independent #News #Media #St...,...,36.701463,-118.755997,NaN,United States,North America,California,CA,2020-10-21 00:00:02.071309132,en,"[censorship, hunterbiden, biden, bidenemail, b..."
2,2020-10-15 00:00:25,-2147483648,"In 2020, #NYPost is being #censorship #CENSORE...",0,0,Twitter for iPhone,19940334,Change Illinois | Biden will increase taxes by...,changeillinois,"Illinois, home of Lincoln and Reagan, used to ...",...,41.875562,-87.624421,Chicago,United States,North America,Illinois,IL,2020-10-21 00:00:03.106963698,en,"[nypost, censorship, censor, twitter, manipul,..."
3,2020-10-15 00:01:23,-2147483648,"Comments on this? ""Do Democrats Understand how...",0,0,Twitter Web App,101659277,John Ubaldi,ubaldireports,Just Facts... No Fiction on domestic and globa...,...,27.947760,-82.458444,Tampa,United States,North America,Florida,FL,2020-10-21 00:00:08.803063811,en,"[comment, democrat, understand, ruthless, chin..."
4,2020-10-15 00:01:42,-2147483648,In an effort to find the truth about allegatio...,1,0,Twitter Web App,-2147483648,The Big Homie,_dbrown_23,#YAHDOG Posse's Top Guy.\n@illinivideo\n@Illin...,...,40.116420,-88.243383,NaN,United States,North America,Illinois,IL,2020-10-21 00:00:10.874372943,en,"[effort, find, truth, alleg, twitter, isnt, al..."


In [37]:
biden_cleaned.shape

(3892, 23)

In [36]:
biden_cleaned.country.unique()

array(['United States', 'United Kingdom', 'Poland', 'China', 'Spain',
       'Egypt', 'Russia', 'Georgia', 'Canada', 'Jersey', 'Australia',
       'Mongolia', 'Netherlands', 'New Zealand', 'The Netherlands',
       'Vietnam', 'Costa Rica', 'Singapore', 'Austria', 'Greece',
       'Malaysia', 'Japan', 'South Korea', 'Turkey', 'India', 'Sweden',
       'Slovenia', 'Papua New Guinea', 'Venezuela', 'Albania', 'Peru',
       'United Arab Emirates', 'Colombia', 'Iran', 'Germany', 'Cuba',
       'Saudi Arabia', 'Italy', 'France', 'Switzerland', 'Tunisia',
       'Philippines', 'Bangladesh', 'Mexico', 'Chile', 'Guyana',
       'Finland', 'Cambodia', 'Pakistan', 'Slovakia', 'Luxembourg',
       'Indonesia', 'South Africa', 'Nigeria', 'Belgium',
       'Czech Republic', 'Sierra Leone', 'Uganda', 'Ireland', 'Taiwan',
       'Lebanon', 'Hungary', 'Azerbaijan', 'Hong Kong', 'Cyprus', 'Fiji',
       'Norway', 'Bahrain', 'Israel', 'Estonia', 'Brazil', 'Liberia',
       'Kenya', 'Nepal', 'Armenia', 'L

In [38]:
print(trump_cleaned.shape, biden_cleaned.shape)

(4283, 23) (3892, 23)


In [39]:
#concat both
df_all = pd.concat([trump_cleaned, biden_cleaned])
df_all.shape

(8175, 23)

In [40]:
df_all.to_excel('sample_cleaned.xlsx')